## Primer inciso que pide elegir activos

In [43]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import rbf_kernel
import matplotlib.pyplot as plt
import yfinance as yf
from IPython.display import display, Markdown
%matplotlib inline

In [44]:
tickers=['AMZN','WMT','TSLA','AAPL','MSFT']

f_inicial='2020-01-01'
f_final='2024-01-01'

closes=yf.download(tickers,f_inicial,f_final)['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [45]:
closes

Ticker,AAPL,AMZN,MSFT,TSLA,WMT
Date,,,,,
2020-01-02 00:00:00+00:00,72.796013,94.900497,153.630676,28.684000,36.864079
2020-01-03 00:00:00+00:00,72.088303,93.748497,151.717712,29.534000,36.538631
2020-01-06 00:00:00+00:00,72.662720,95.143997,152.109894,30.102667,36.464256
2020-01-07 00:00:00+00:00,72.320984,95.343002,150.722977,31.270666,36.126419
2020-01-08 00:00:00+00:00,73.484352,94.598503,153.123734,32.809334,36.002449
...,...,...,...,...,...
2023-12-22 00:00:00+00:00,192.656174,153.419998,371.799713,252.539993,51.712521
2023-12-26 00:00:00+00:00,192.108841,153.410004,371.879150,256.609985,51.633293
2023-12-27 00:00:00+00:00,192.208374,153.339996,371.293518,261.440002,52.118561


In [46]:
ret = closes.pct_change().dropna()
mu = ret.mean()
mu # vector mu

Ticker
AAPL    0.001187
AMZN    0.000750
MSFT    0.001095
TSLA    0.003070
WMT     0.000454
dtype: float64

In [47]:
sigma = ret.cov()
sigma # matriz de covarianza

Ticker,AAPL,AMZN,MSFT,TSLA,WMT
Ticker,,,,,
AAPL,0.000447,0.000313,0.000338,0.000463,0.000126
AMZN,0.000313,0.000564,0.000331,0.000463,0.000109
MSFT,0.000338,0.000331,0.000422,0.000416,0.000124
TSLA,0.000463,0.000463,0.000416,0.001841,0.000102
WMT,0.000126,0.000109,0.000124,0.000102,0.000221


In [48]:
sigma_inv = np.linalg.inv(sigma)
np.round(pd.DataFrame(sigma_inv),2) # matriz de covarianza inversa

,0,1,2,3,4
0,6383.39,-746.06,-3720.76,-525.38,-957.11
1,-746.06,3538.67,-1870.33,-270.52,-147.24
2,-3720.76,-1870.33,7267.47,-184.84,-931.58
3,-525.38,-270.52,-184.84,775.42,180.00
4,-957.11,-147.24,-931.58,180.00,5576.78


In [49]:
ones = np.ones(len(mu))
ones # vector de unos

array([1., 1., 1., 1., 1.])

In [50]:
w = (sigma_inv.dot(ones))/(ones.T.dot(sigma_inv).dot(ones))
w # vector de pesos para portafolio de mínima varianza

array([ 0.08357244,  0.0971338 ,  0.10780546, -0.00487524,  0.71636353])

In [51]:
w_df = pd.DataFrame(w, index=closes.columns, columns=['w'])
w_df.T # pesos en DataFrame

Ticker,AAPL,AMZN,MSFT,TSLA,WMT
w,0.083572,0.097134,0.107805,-0.004875,0.716364


In [52]:
rp = w.T.dot(mu)
rp # rendimiento esperado del portafolio de mínima varianza diario

0.0006001983508463384

In [53]:
mu

Ticker
AAPL    0.001187
AMZN    0.000750
MSFT    0.001095
TSLA    0.003070
WMT     0.000454
dtype: float64

## Segundo inciso que pide elegir activos

In [54]:
U = np.array([mu, ones]).T
U

array([[1.18656757e-03, 1.00000000e+00],
       [7.49814311e-04, 1.00000000e+00],
       [1.09462736e-03, 1.00000000e+00],
       [3.06974350e-03, 1.00000000e+00],
       [4.53904469e-04, 1.00000000e+00]])

In [55]:
M = U.T.dot(sigma_inv).dot(U)
pd.DataFrame(M)

,0,1
0,0.006208,3.117476
1,3.117476,5194.076071


In [56]:
M_inv = np.linalg.inv(M)
pd.DataFrame(M_inv)

,0,1
0,230.604353,-0.138408
1,-0.138408,0.000276


In [57]:
u = np.array([[rp],
              [1]])
u

array([[6.00198351e-04],
       [1.00000000e+00]])

In [58]:
w2 = sigma_inv.dot(U).dot(M_inv).dot(u)
w2

array([[ 0.08357244],
       [ 0.0971338 ],
       [ 0.10780546],
       [-0.00487524],
       [ 0.71636353]])

In [59]:
w2_df = pd.DataFrame(w2, index=closes.columns, columns=['w'])
w2_df.T # pesos en DataFrame

Ticker,AAPL,AMZN,MSFT,TSLA,WMT
w,0.083572,0.097134,0.107805,-0.004875,0.716364
